In [ ]:
from credentials import PINECONE_API_KEY
from pinecone import Pinecone
from json import dumps
from vertexai.preview.generative_models import GenerativeModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.language_models import TextEmbeddingInput
from langchain.text_splitter import SpacyTextSplitter

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("infofin")

index.delete(delete_all=True)

In [ ]:
with open("./Memoria_BCH_2022_interactiva-10-05.md", "r") as f:
    markdown_text = f.read()

In [ ]:
text_splitter = SpacyTextSplitter(
    pipeline="es_dep_news_trf",
    chunk_size=500,
    chunk_overlap=50
)

docs = text_splitter.split_text(markdown_text)
print(len(docs))

In [5]:
gecko = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

chunk_number = 0
for doc in docs:
    text_embedding_input = TextEmbeddingInput(
        text=doc,
        task_type="RETRIEVAL_DOCUMENT"
    )
    vector = gecko.get_embeddings([text_embedding_input])
    index.upsert([(str(chunk_number), vector[0].values, {"text": doc})])
    chunk_number = chunk_number + 1

In [6]:
gemini = GenerativeModel("gemini-pro")

def ask_gemini(question: str, print_prompt=False) -> str:
    
    text_embedding_input = TextEmbeddingInput(
        text=question,
        task_type="RETRIEVAL_QUERY"
    )
    
    vector = gecko.get_embeddings([text_embedding_input])
    
    matches = index.query(
        vector=vector[0].values,
        top_k=5,
        include_metadata=True
    )

    # " Si la respuesta no se encuentra en el documento, indica que la información no está en el documento." \
    # " Siempre debes responder en primera persona como un empleado del banco." \
    # " Responde en un párrafo o menos." \
    prompt = \
        "Contexto: Eres experto en responder preguntas sobre el Banco de Chile." \
        " En el siguiente documento se encuentran los extractos que debes usar para responder." \
        " Siempre debes responder en primera persona como un empleado del banco." \
        " Siempre debes responder en primera persona como un empleado del banco." \
        f"\nDocumento: {dumps(matches.to_dict(), ensure_ascii=False)}" \
        f"\nPregunta: ¿Quién es el Gerente General de Banco de Chile?" \
        f"\nRespuesta: Eduardo Ebenperger Orrego." \
        f"\nPregunta: ¿Cuántos años tiene Banco de Chile operando en el país?" \
        f"\nRespuesta: 129 años." \
        f"\nPregunta: ¿Cuál es la misión del Banco de Chile?" \
        f"\nRespuesta: Contribuir al desarrollo del país y al progreso de las personas, mejorando su calidad de vida sin comprometer el bienestar de las futuras generaciones. Impulsamos el crecimiento económico de manera equilibrada con el medio ambiente y en igualdad de oportunidades." \
        f"\nPregunta: {question}" \
        f"\nRespuesta:"
    
    if print_prompt:
        print(prompt)
    
    gemini_response = gemini.generate_content(prompt, stream=False)
    
    try:
        return gemini_response.text
    except:
        return "Error!"

In [7]:
ask_gemini("¿Cuántas sucursales hay en Tarapacá?", print_prompt=False)

'4.'

In [8]:
from vertexai.language_models import TextGenerationModel

unicorn = TextGenerationModel.from_pretrained("text-unicorn@001")

def ask_unicorn(question: str, print_prompt=False) -> str:
    
    text_embedding_input = TextEmbeddingInput(
        text=question,
        task_type="RETRIEVAL_QUERY"
    )
    
    vector = gecko.get_embeddings([text_embedding_input])
    

    matches = index.query(
        vector=vector[0].values,
        top_k=5,
        include_metadata=True
    )

    # " Si la respuesta no se encuentra en el documento, indica que la información no está en el documento." \
    # " Siempre debes responder en primera persona como un empleado del banco." \
    # " Responde en un párrafo o menos." \
    prompt = \
        "Contexto: Eres experto en responder preguntas sobre el Banco de Chile." \
        " En el siguiente documento se encuentran los extractos que debes usar para responder." \
        " Siempre debes responder en primera persona como un empleado del banco." \
        " Siempre debes responder en primera persona como un empleado del banco." \
        f"\nDocumento: {dumps(matches.to_dict(), ensure_ascii=False)}" \
        f"\nPregunta: ¿Quién es el Gerente General de Banco de Chile?" \
        f"\nRespuesta: Eduardo Ebenperger Orrego." \
        f"\nPregunta: ¿Cuántos años tiene Banco de Chile operando en el país?" \
        f"\nRespuesta: 129 años." \
        f"\nPregunta: ¿Cuál es la misión del Banco de Chile?" \
        f"\nRespuesta: Contribuir al desarrollo del país y al progreso de las personas, mejorando su calidad de vida sin comprometer el bienestar de las futuras generaciones. Impulsamos el crecimiento económico de manera equilibrada con el medio ambiente y en igualdad de oportunidades." \
        f"\nPregunta: {question}" \
        f"\nRespuesta:"
    
    if print_prompt:
        print(prompt)
    
    unicorn_response = unicorn.predict(prompt)
    
    try:
        return unicorn_response.text
    except:
        return "Error!"

In [9]:
ask_unicorn("¿Cuántas sucursales hay en Tarapacá?", print_prompt=False)

'En la región de Tarapacá hay 4 sucursales.'

In [ ]:
with open("./questions.txt", "r") as f:
    questions = f.read()
    questions = questions.split("\n")

with open("./questions_raw.txt", "r") as f:
    raws = f.read()
    raws = raws.split("\n")

csv_format = "{},\"{}\",\"{}\""

question_number = 1
for question, raw in zip(questions, raws):
    answer = ask_gemini(question, print_prompt=False)
    print(csv_format.format(question_number, raw, answer))
    question_number = question_number + 1

In [ ]:
with open("./questions.txt", "r") as f:
    questions = f.read()
    questions = questions.split("\n")

with open("./questions_raw.txt", "r") as f:
    raws = f.read()
    raws = raws.split("\n")

csv_format = "{},\"{}\",\"{}\""

for question, raw in zip(questions, raws):
    answer = ask_unicorn(question, print_prompt=False)
    print(csv_format.format(question_number, raw, answer))